<a href="https://colab.research.google.com/github/Waleed123/Kaggle_Competition_new/blob/main/Kagglecompetition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("test")